In [2]:
import sqlalchemy
import pymysql
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import random
import numpy as np
import os
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
class Project():
    def __init__(self, host = "localhost", port = 3306, user = "root", password = "wyjdsg1998524"):
        # make the connection to mysql using pymysql
        self.port = port
        self.host = host
        self.user = user
        self.password = password
        self.conn = pymysql.connect(host = host,
                               port = port,
                               user = user,
                               password = password,
                               charset = "utf8")
        # create a cursor, which we can use to execute sql 
        self.cursor = self.conn.cursor()
        # create database project
        sql = "CREATE DATABASE IF NOT EXISTS project;"
        self.cursor.execute(sql)
        sql = "USE project;"
        self.cursor.execute(sql)
        
        # prepare sqlalchemy for dataframe operation
        db_connection_str = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'\
                            .format(user = self.user, password = self.password, host = self.host, \
                            port = str(self.port), db = 'storage0')
        self.db_connection  = sqlalchemy.create_engine(db_connection_str)
        
        
        # create database for file content storage
        sql = "CREATE DATABASE IF NOT EXISTS storage0;"
        self.cursor.execute(sql)
        sql = "CREATE DATABASE IF NOT EXISTS storage1;"
        self.cursor.execute(sql)
        sql = "CREATE DATABASE IF NOT EXISTS storage2;"
        self.cursor.execute(sql)
        sql = "CREATE DATABASE IF NOT EXISTS storage3;"
        self.cursor.execute(sql)
        
        
        # Table metadata:
        # schema: (meta_id, filename, filetype)
        # meta_id is what we are used as inumber
        # filename is the name of the node
        # filetype is the type of the node. In this case, we have "file" or "directory"
        sql = "CREATE TABLE IF NOT EXISTS metadata (meta_id INT, \
                                                    filename VARCHAR(100), \
                                                    fileType VARCHAR(100), \
                                                    hasFile INT,\
                                                    PRIMARY KEY (meta_id, filename)\
                                                    );"
        self.cursor.execute(sql)
        
        # Table directory:
        # schema: (parent, child)
        # parent/child
        sql = "CREATE TABLE IF NOT EXISTS directory (parent INT, child INT);"
        self.cursor.execute(sql)
        
        # Table partitions
        # schema: (filename_id, partition)
        sql = "CREATE TABLE IF NOT EXISTS partitions (filename INT, part VARCHAR(1000), storage VARCHAR(1000));"
        self.cursor.execute(sql)
        
        # init Table metadata
        try:
            sql = "INSERT INTO metadata VALUES (0, 'root', 'directory', 0);"
            self.cursor.execute(sql)
            self.conn.commit() # must have this line together with insert sql
        except:
            pass
        
        # current_node is used to record the current node
        self.current_node = 0
        
        # max_node_id is used to infer the "actual" directory node or file node
        sql = "SELECT MAX(meta_id) from metadata;"
        self.cursor.execute(sql)
        self.max_node_id = self.cursor.fetchall()
        
        # min_node_id is used to infer the "fake" partition file position node
        sql = "SELECT MAX(meta_id) from metadata;"
        self.cursor.execute(sql)
        self.max_node_id = self.cursor.fetchall()[0][0]
        print("Initialization success!!!")
        
        
    # prepare sqlalchemy for dataframe operation
    def db_connection_change(self, db = 'storage0'):
        db_connection_str = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'\
                            .format(user = self.user, password = self.password, host = self.host, \
                            port = str(self.port), db = db)
        self.db_connection  = sqlalchemy.create_engine(db_connection_str)
    
    
    
    
    def mkdir(self, dir_path):
        sql = "USE project;"
        self.cursor.execute(sql)
        if dir_path.endswith('.json'):
            print('Please create a directory, rather than a file')
        else:
            change = False # record whether the dir exists
            path_nodes = dir_path.split('/')
            sql = "SELECT filename FROM metadata a JOIN (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from (SELECT meta_id, filename FROM metadata a JOIN \
                                (select child from directory where parent = %s) b ON a.meta_id = b.child) c \
                                where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
                else:
                    self.max_node_id += 1
                    
                    # insert metadata into Table metadata
                    sql = "INSERT INTO metadata VALUES (%s, %s, %s, %s);"
                    self.cursor.execute(sql, (self.max_node_id, node, 'directory', 0))
                    self.conn.commit() # must have this line together with insert sql
                    
                    # insert metadata into Table directory
                    sql = "INSERT INTO directory VALUES (%s, %s);"
                    self.cursor.execute(sql, (parent_node_id, self.max_node_id))
                    self.conn.commit() # must have this line together with insert sql
                    
                    parent_node_id = self.max_node_id
                    change = True
                    
                    
            if not change:
                print('The directory ' + dir_path + ' already exists!')
            else:
                print('The directory ' + dir_path + '  is successfully made!')
        
        return 
    
    
    
    
    
    def ls(self, dir_path):
        sql = "USE project;"
        self.cursor.execute(sql)
        if dir_path.endswith('.json'):
            print('Please search for a directory, rather than a file')
            return
        if dir_path[0] != '/':
            print('Please input an available directory path')
            return
        elif dir_path == '/':
            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            print('\t'.join(node_children_node))
            return
        else:
            path_nodes = dir_path.split('/')
            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from \
                        (SELECT meta_id, filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                        ON a.meta_id = b.child \
                        ) c \
                        where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
                    
                    #print(node_children_node)
                else:
                    print('The directory does not exist!')
                    return 
            print('\t'.join(node_children_node))
            return
        
    
    
    
    def put(self, file_name, file_path, k = 5, sort_key=None):
        '''
        upload a file from file_name on the local machine to the EDFS file_path
        '''
        try:
            k = int(k)
        except:
            print("Please input an integer k!")
            return
        
        sql = "USE project;"
        self.cursor.execute(sql)
        if k < 1:
            print('k partition must be larger than 1')
            return 
        if not file_name.endswith('.json'):
            print('Please upload a file')
            return 
        if file_path.endswith('.json'):
            print('Please choose a directory')
            return
        if not os.path.exists(file_name):
            print('The file does not exist!')
            return
        else:
            # judge whether the similar file already exists in the fils system
            filename = file_name.split('/')[-1]
            
            sql = "SELECT meta_id from metadata \
                    where filename = %s and fileType = 'file';"
            self.cursor.execute(sql, (filename))
            fetchall = self.cursor.fetchall()
            if len(fetchall) != 0:
                dire_path = '/' + filename
                print('The same file already exists in the file system')
                parent_node_id = fetchall[0][0]
                while parent_node_id != 0:
                    sql = "SELECT a.meta_id, a.filename FROM metadata a JOIN directory b on a.meta_id = b.parent and b.child = %s;"
                    self.cursor.execute(sql, (parent_node_id))
                    fetchall = self.cursor.fetchall()
                    parent_node_id = fetchall[0][0]
                    if parent_node_id == 0:
                        break
                    _filename = fetchall[0][1]
                    dire_path = '/' + _filename + dire_path
                print("The path of the same file is:", dire_path)
            else:
                # find the parent node id of the file_path
                path_nodes = file_path.split('/')
                sql = "SELECT filename FROM metadata a JOIN \
                        (select child from directory where parent = 0) b \
                        ON a.meta_id = b.child;"
                self.cursor.execute(sql)
                node_children_node = [i[0] for i in self.cursor.fetchall()]
                parent_node_id = 0
                for node in path_nodes[1:]:
                    # print(node)
                    if node in node_children_node:
                        sql = "SELECT meta_id from \
                            (SELECT meta_id, filename FROM metadata a JOIN \
                                (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child \
                            ) c \
                            where filename = %s;"
                        self.cursor.execute(sql, (parent_node_id, node))
                        parent_node_id = self.cursor.fetchall()[0][0]
                        sql = "SELECT filename FROM metadata a JOIN \
                                (select child from directory where parent = %s) b \
                                ON a.meta_id = b.child;"
                        self.cursor.execute(sql, (parent_node_id))
                        node_children_node = [i[0] for i in self.cursor.fetchall()]

                        #print(node_children_node)
                    else:
                        print('The directory does not exist!')
                        return 
                
                
                
                
                if 'business' in filename:
                    '''
                    # Deal with the json annd convert it to dataframe!!!!!
                    '''
                    business = []
                    with open(file_name,encoding='utf-8') as f1:
                        for f in f1:
                            business.append(f)
                    business1 = [json.loads(x) for x in business]
                    df_b = pd.json_normalize(business1)
                    cls1 = list(df_b)
                    print('Acceptable sort keys are', cls1)
                    sort_key = input('Please choose one of the features for partition: ')
                    
                    '''
                    # e.g. sort df_b on business_id and partition it by business_id
                    '''
                    while True:
                        try:
                            df_b = df_b.sort_values(sort_key).reset_index(drop = True)
                            break
                        except: # default sort by business_id
                            print('Please input the correct feature name')
                    
                    
                   

                    
                elif 'review' in filename:
                    '''
                    # Deal with the json annd convert it to dataframe!!!!!
                    '''
                    review = []
                    with open(file_name, encoding = 'utf-8') as g1:
                        for g in g1:
                            review.append(g)
                    review1 = [json.loads(x) for x in review]
                    df_b = pd.json_normalize(review1)
                    cls2 = list(df_b)
                    print('Acceptable sort keys are', cls2)
                    sort_key = input('Please choose one of the features for partition: ')
                    while True:
                        try:
                            df_b = df_b.sort_values(sort_key).reset_index(drop = True)
                            break
                        except: # default sort by business_id
                            print('Please input the correct feature name')
                    
                
                
                elif 'user' in filename:
                    '''
                    # Deal with the json annd convert it to dataframe!!!!!
                    '''
                    user = []
                    with open(file_name, encoding = 'utf-8') as h1:
                        for h in h1:
                            user.append(h)
                    user1 = [json.loads(x) for x in user]
                    df_b = pd.json_normalize(user1)
                    df_b.drop(['friends'], inplace = True, axis = 1)
                    cls3 = list(df_b)
                    print('Acceptable sort keys are', cls3)
                    sort_key = input('Please choose one of the features for partition: ')
                    
                    while True:
                        try:
                            df_b = df_b.sort_values(sort_key).reset_index(drop = True)
                            break
                        except: # default sort by business_id
                            print('Please input the correct feature name')
                else:
                    print("Please upload the files about business, user or review_train")
                    return
                
                # insert metadata into Table metadata
                self.max_node_id += 1
                file_node_id = self.max_node_id
                sql = "INSERT INTO metadata VALUES (%s, %s, %s, %s);"
                self.cursor.execute(sql, (file_node_id, filename, 'file', 0))
                self.conn.commit() # must have this line together with insert sql
                
                # insert metadata into Table directory
                sql = "INSERT INTO directory VALUES (%s, %s);"
                self.cursor.execute(sql, (parent_node_id, file_node_id))
                self.conn.commit() # must have this line together with insert sql    
                    
                
                for i in tqdm(range(k), desc = "Creating Partition"):
                    # storage the partitioned file into distributed database
                    if i != k - 1:
                        for n in tqdm(np.random.choice(4, 3, replace = False), desc = 'Creating replica'): # replica number = 3
                            self.db_connection_change('storage' + str(n))
                            sql = "USE storage" + str(n) + ";"
                            self.cursor.execute(sql)
                            df_b_partition = df_b.iloc[df_b.shape[0] // k * i: df_b.shape[0] // k * (i + 1), :]
                            df_b_partition.to_sql(filename.split('.json')[0] + '_partition{}'.format(i), self.db_connection, index = True, index_label = 'idx')
                            # insert metadata into Table partitions
                            sql = "USE project;"
                            self.cursor.execute(sql)
                            sql = "INSERT INTO partitions VALUES (%s, %s, %s);"
                            self.cursor.execute(sql, (file_node_id, filename.split('.json')[0] + '_partition{}'.format(i), "storage" + str(n)))
                            self.conn.commit() # must have this line together with insert sql
                    else:
                        for n in tqdm(np.random.choice(4, 3, replace = False), desc = 'Creating replica'):
                            # storage the partitioned file into distributed database
                            self.db_connection_change('storage' + str(n))
                            sql = "USE storage" + str(n) + ";"
                            self.cursor.execute(sql)
                            df_b_partition = df_b.iloc[df_b.shape[0] // k * (k - 1):, :]
                            df_b_partition.to_sql(filename.split('.json')[0] + '_partition{}'.format(k - 1), self.db_connection, index = True, index_label = 'idx')
                            # insert metadata into Table partitions
                            sql = "USE project;"
                            self.cursor.execute(sql)
                            sql = "INSERT INTO partitions VALUES (%s, %s, %s);"
                            self.cursor.execute(sql, (file_node_id, filename.split('.json')[0] + '_partition{}'.format(k - 1), "storage" + str(n)))
                            self.conn.commit() # must have this line together with insert sql
                    
                
                
                

                
                # update the file name for each directories
                path_nodes = file_path.split('/')
                sql = "UPDATE metadata SET hasFile = hasFile + 1 where meta_id = 0"
                self.cursor.execute(sql)
                self.conn.commit()
                
                sql = "SELECT filename FROM metadata a JOIN \
                        (select child from directory where parent = 0) b \
                        ON a.meta_id = b.child;"
                self.cursor.execute(sql)
                node_children_node = [i[0] for i in self.cursor.fetchall()]
                parent_node_id = 0
                for node in path_nodes[1:]:
                    # print(node)
                    if node in node_children_node:
                        sql = "SELECT meta_id from \
                            (SELECT meta_id, filename FROM metadata a JOIN \
                                (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child \
                            ) c \
                            where filename = %s;"
                        self.cursor.execute(sql, (parent_node_id, node))
                        parent_node_id = self.cursor.fetchall()[0][0]
                        # print(parent_node_id)
                        sql = "UPDATE metadata SET hasFile = hasFile + 1 WHERE meta_id = %s"
                        self.cursor.execute(sql, (parent_node_id))
                        self.conn.commit()
                        
                        
                        sql = "SELECT filename FROM metadata a JOIN \
                                (select child from directory where parent = %s) b \
                                ON a.meta_id = b.child;"
                        self.cursor.execute(sql, (parent_node_id))
                        node_children_node = [i[0] for i in self.cursor.fetchall()]
                    
            
                print(file_name + ' uploaded successfully!')
    
    
    def rm(self, dir_path):
        '''
        delete a file from the dir_path
        '''
        sql = "USE project;"
        self.cursor.execute(sql)
        
        if not dir_path.endswith('.json'):
            print('Please search for a file to delete, rather than a directory')
            return 
        else:
            # get the file's node id
            path_nodes = dir_path.split('/')
            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from \
                        (SELECT meta_id, filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                        ON a.meta_id = b.child \
                        ) c \
                        where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
                    
                    #print(node_children_node)
                else:
                    print('The file does not exist!')
                    return 
            
            # get the partitions of the files
            filename = node
            file_node_id = parent_node_id
            # get how many partitions of this file
            sql = "SELECT DISTINCT part from partitions where filename = %s"
            self.cursor.execute(sql, (file_node_id))
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            for partition_table_name in node_children_node:
                sql = "USE project;"
                self.cursor.execute(sql)
                sql = "SELECT DISTINCT storage from partitions where part = %s"
                self.cursor.execute(sql, (partition_table_name))
                storage_positions = [i[0] for i in self.cursor.fetchall()]
                for storage in storage_positions:
                    
                    sql = "USE " + storage + ";"
                    self.cursor.execute(sql)
                    sql = "DROP TABLE {table_name}".format(table_name = partition_table_name)
                    self.cursor.execute(sql)
                    self.conn.commit()
    
            
            # update the "hasFile" features in table metadata
            path_nodes = dir_path.split('/')
            sql = "USE project;"
            self.cursor.execute(sql)
            sql = "UPDATE metadata SET hasFile = hasFile - 1 where meta_id = 0"
            self.cursor.execute(sql)
            self.conn.commit()

            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from \
                        (SELECT meta_id, filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                        ON a.meta_id = b.child \
                        ) c \
                        where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "UPDATE metadata SET hasFile = hasFile - 1 WHERE meta_id = %s"
                    self.cursor.execute(sql, (parent_node_id))
                    self.conn.commit()


                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
            
            
            # delete metadata for the file
            sql = "USE project;"
            self.cursor.execute(sql)
            sql = "DELETE FROM partitions WHERE filename = %s"
            self.cursor.execute(sql, (file_node_id))
            self.conn.commit()
            sql = "DELETE FROM directory WHERE child = %s"
            self.cursor.execute(sql, (file_node_id))
            self.conn.commit()
            sql = "DELETE FROM metadata WHERE meta_id = %s"
            self.cursor.execute(sql, (file_node_id))
            self.conn.commit()
        
            print('The file has been deleted')
    
    
    
    
    
    def cat(self, dir_path, offset = 0, limit = 1):
        try:
            offset = int(offset)
            limit = int(limit)
        except:
            print("Please input an integer offset and limit!")
            return
        '''
        show the slides of the file.
        The files we use will be too large and will not be suitable for show
        so the user can only see the slides of the file from line offset to line (limit + offset)
        '''
        sql = "USE project;"
        self.cursor.execute(sql)
        if limit > 20:
            print('please not ask to display larger than 20 lines of the data')
            return
        if offset < 0 or limit < 0:
            print('Please input offset and limit which are > 0')
            return
        if not dir_path.endswith('.json'):
            print('Please search for a file, rather than a directory')
            return
        else:
            # get the file's node id
            path_nodes = dir_path.split('/')
            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from \
                        (SELECT meta_id, filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                        ON a.meta_id = b.child \
                        ) c \
                        where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
                    
                    #print(node_children_node)
                else:
                    print('The file does not exist!')
                    return 
            
            # get the partitions of the files
            filename = node
            file_node_id = parent_node_id
            # get how many partitions of this file
            sql = "SELECT DISTINCT part from partitions where filename = %s"
            self.cursor.execute(sql, (file_node_id))
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            res = pd.DataFrame()
            for partition_table_name in node_children_node:
                sql = "USE project;"
                self.cursor.execute(sql)
                sql = "SELECT DISTINCT storage from partitions where part = %s"
                self.cursor.execute(sql, (partition_table_name))
                storage_positions = [i[0] for i in self.cursor.fetchall()]
                for storage in storage_positions:
                    try:
                        sql = "USE " + storage + ";"
                        self.cursor.execute(sql)
                        sql = "SELECT * from {table_name} \
                            where idx < {offset} + {limit} and idx >= {offset}".format(table_name = partition_table_name, limit = str(limit), offset = str(offset))
                        res = pd.concat([res, pd.read_sql_query(sql, self.conn)])
                        break
                    except:
                        continue
                    
            res.reset_index(drop = True, inplace = True)
            res.drop('idx', axis = 1, inplace = True)
            print(res)
            
    
    
    
    
    
    def rmdir(self, dir_path):
        '''
        If dir_path has files in it, then it will report one of the files it contains and not operate rm
        If there is no files in it, then it will be deleted, and all of its child directory will be deleted either.
        '''
        sql = "USE project;"
        self.cursor.execute(sql)
        
        if dir_path.endswith('.json'):
            print('Please search for a directory to delete, rather than a file')
            return 
        else:
            # get the directory's node id
            path_nodes = dir_path.split('/')
            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from \
                        (SELECT meta_id, filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                        ON a.meta_id = b.child \
                        ) c \
                        where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
                else:
                    print('The directory does not exist!')
                    return 
            
            
            dir_node_id = parent_node_id
            # print(dir_node_id)
            # find whether the directory contains files
            sql = "SELECT hasFile from metadata where meta_id = %s"
            self.cursor.execute(sql, (dir_node_id))
            hasFile = self.cursor.fetchall()[0][0]
            res = dir_path
            if hasFile != 0:
                print('The dir has files in it, please check:', end = ' ')
                while True:
                    sql = "SELECT meta_id, filename from \
                            (SELECT meta_id, filename, hasFile FROM metadata a JOIN \
                                (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child \
                            ) c \
                            where hasFile > 0;"
                    self.cursor.execute(sql, (dir_node_id))
                    fetchall = self.cursor.fetchall()
                    if len(fetchall) != 0:
                        dir_node_id = fetchall[0][0]
                        filename = fetchall[0][1]
                        res = res + '/' + filename
                    else:
                        break
                print(res)
            else:
                # find all the nodes we need to delete
                delete_nodes = []
                dir_node_id = parent_node_id
                delete_nodes.append(dir_node_id)
                # delete metadata for the file
                sql = "USE project;"
                self.cursor.execute(sql)
                tmp_node_lst = [dir_node_id]
                while True:
                    tmp_lst = []
                    sql = "SELECT child FROM directory WHERE parent = %s;"
                    for node_id in tmp_node_lst:
                        self.cursor.execute(sql, (node_id))
                        fetchall = self.cursor.fetchall()
                        if len(fetchall) != 0:
                            fetchall = [i[0] for i in fetchall]
                            delete_nodes.extend(fetchall)
                            tmp_lst.extend(fetchall)
                    tmp_node_lst = tmp_lst
                    if len(tmp_node_lst) == 0:
                        break
                
                delete_nodes = [str(i) for i in delete_nodes]
                delete_nodes = '(' + ','.join(delete_nodes) + ')'
                
                # delete the nodes
                sql = "DELETE FROM directory WHERE parent in {delete_nodes};".format(delete_nodes = delete_nodes)
                self.cursor.execute(sql)
                self.conn.commit()
                sql = "DELETE FROM directory WHERE child in {delete_nodes};".format(delete_nodes = delete_nodes)
                self.cursor.execute(sql)
                self.conn.commit()
                sql = "DELETE FROM metadata WHERE meta_id in {delete_nodes};".format(delete_nodes = delete_nodes)
                self.cursor.execute(sql)
                self.conn.commit()
                
                print('The directory has been deleted')

            
            
            
            
            
    
    def getPartitionLocations(self, dir_path):
        '''
        return a list contains (DataNode, Table name) for each partitions
        '''
        sql = "USE project;"
        self.cursor.execute(sql)
        
        if not dir_path.endswith('.json'):
            print('Please search for a file, rather than a directory')
            return
        else:
            # get the file's node id
            path_nodes = dir_path.split('/')
            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from \
                        (SELECT meta_id, filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                        ON a.meta_id = b.child \
                        ) c \
                        where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
                    
                    #print(node_children_node)
                else:
                    print('The file does not exist!')
                    return 
            
            # get the partitions of the files
            filename = node
            file_node_id = parent_node_id
            # get how many partitions of this file
            sql = "select part, storage from \
                    (SELECT part, storage, rank() over(partition by part order by storage asc) as rak\
                    from partitions where filename = %s) a\
                   where rak = 1;"
            self.cursor.execute(sql, (file_node_id))
            node_children_node = [i for i in self.cursor.fetchall()]
            for pair in node_children_node:
                print('Partition_name: ' + pair[0] + '-> DataNode: ' + pair[1])
            return node_children_node
    
    
    
    
    
    def readPartition(self, dir_path, k):
        '''
        return NameNode(database), Table name, partition_file in pd.DataFrame format
        '''
        try:
            k = int(k)
        except:
            print("Please input an integer k!")
            return
        sql = "USE project;"
        self.cursor.execute(sql)
        
        if not dir_path.endswith('.json'):
            print('Please search for a file, rather than a directory')
            return
        else:
            # get the file's node id
            path_nodes = dir_path.split('/')
            sql = "SELECT filename FROM metadata a JOIN \
                    (select child from directory where parent = 0) b \
                    ON a.meta_id = b.child;"
            self.cursor.execute(sql)
            node_children_node = [i[0] for i in self.cursor.fetchall()]
            parent_node_id = 0
            for node in path_nodes[1:]:
                # print(node)
                if node in node_children_node:
                    sql = "SELECT meta_id from \
                        (SELECT meta_id, filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                        ON a.meta_id = b.child \
                        ) c \
                        where filename = %s;"
                    self.cursor.execute(sql, (parent_node_id, node))
                    parent_node_id = self.cursor.fetchall()[0][0]
                    sql = "SELECT filename FROM metadata a JOIN \
                            (select child from directory where parent = %s) b \
                            ON a.meta_id = b.child;"
                    self.cursor.execute(sql, (parent_node_id))
                    node_children_node = [i[0] for i in self.cursor.fetchall()]
                    
                    #print(node_children_node)
                else:
                    print('The file does not exist!')
                    return 
            
            # get the partitions of the files
            filename = node
            file_node_id = parent_node_id
            partition_file_name = filename.split('.json')[0] + '_partition{}'.format(k)
            sql = "SELECT storage from partitions where filename = %s and part = %s;"
            self.cursor.execute(sql, (file_node_id, partition_file_name))
            res = pd.DataFrame()
            for replica in range(3):
                try:
                    storage = self.cursor.fetchall()[replica][0]
                    sql = "USE " + storage + ";"
                    self.cursor.execute(sql)
                    sql = "SELECT * from " + partition_file_name + ";" 
                    res = pd.concat([res, pd.read_sql_query(sql, self.conn)])
                    break
                except:
                    continue
                    
            # res.reset_index(drop = True, inplace = True)
            # res.drop('idx', axis = 1, inplace = True)
            return storage, partition_file_name, res
        
    
    def mapPartition(self, dir_path, k, func): # example: func: stars>3
        storage = self.readPartition(dir_path,k)[0]
        partition_file_name = self.readPartition(dir_path,k)[1]
        sql1 = "USE " + storage + ";"
        self.cursor.execute(sql1)
        part = self.readPartition(dir_path,k)[2]
        sql2 = "SELECT * from " + partition_file_name + " WHERE " + func +";"
        res = pd.read_sql_query(sql2, self.conn)
        #print(res.head())
        return res
    
    def reduce(self, dir_path, func1, func2, group_key): # example: func1: review_count>10, func2: state, avg(stars), group_key: state
        comb = pd.DataFrame()
        for l in range(len(self.getPartitionLocations(dir_path))):
            comb = pd.concat([comb, self.mapPartition(dir_path, l, func1)])
        sql0 = "USE project;"
        self.cursor.execute(sql0)
        sql1 = "DROP TABLE IF EXISTS temp;"
        self.cursor.execute(sql1)
        self.db_connection_change('project')
        comb.to_sql('temp', self.db_connection, index = True, index_label = 'idy')
        sql = "SELECT " + func2 + " from temp GROUP BY " + group_key +";"
        res = pd.read_sql_query(sql, self.conn)
        #print(res.head())
        return res
        
    
    
    
    
    
    def endconn(self):
        self.conn.close()
    

        
    
demo = Project()
# demo.mkdir('/user/johns/a')
# demo.mkdir('/user/johns/b')
# demo.mkdir('/users/yjs/a')
# demo.mkdir('/users/yjs/b')
# demo.mkdir('/users/yjs/b')
# demo.mkdir('/users/yjsss/a')
# demo.ls('/user/johns')
# demo.ls('/')
# demo.put('business.json', '/user/johns/b', 7, 'unkonwn')
# demo.put('review_train.json', '/user/johns/b')
# demo.put('user.json', '/user')
# demo.cat('/user/user.json', 37710, 20)
# demo.rm('/user/johns/b/business.json')
# demo.rmdir('/users')
# demo.getPartitionLocations('/user/user.json')
# storage, partition_file_name, res = demo.readPartition('/user/user.json', 0)
# demo.mapPartition('/user/johns/b/business.json', 1, 'stars>3')
# demo.reduce('/user/johns/b/business.json', 'review_count>10', 'state, avg(stars)', 'state')
demo.endconn()

Initialization success!!!


In [3]:
def YELP_ANALYSIS_APP():
    print("Welcome to the YELP ANALYSIS system!")
    print("Now you are in the EDFS. Please write your commands below!")
    demo = Project()
    while True:
        command = input(">>> ")
        if command == '':
            continue
        commands = command.split(' ')
        if commands[0] == 'mkdir':
            try:
                demo.mkdir(commands[1])
                continue
            except Exception as e:
                if type(e) == IndexError:
                    print('Please give parameter (dir_path)!')
                else:
                    print(e)
                continue
        elif commands[0] == 'ls':
            try:
                demo.ls(commands[1])
                continue
            except Exception as e:
                if type(e) == IndexError:
                    print('Please give parameter (dir_path)!')
                else:
                    print(e)

                continue
        elif commands[0] == 'put':
            try:
                if len(commands) == 3:
                    demo.put(commands[1], commands[2])
                elif len(commands) == 4:
                    demo.put(commands[1], commands[2], commands[3])
                else:
                    print('Please give parameters (file_path on your local machine, dir_path, k[option])!')
                continue
            except Exception as e:
                if type(e) == IndexError:
                    print('Please give parameters (file_path on your local machine, dir_path, k[option])!')
                else:
                    print(e)
                continue
        elif commands[0] == 'cat':
            try:
                demo.cat(commands[1], commands[2], commands[3])
                continue
            except Exception as e:
                if type(e) == IndexError:
                    print('Please give parameters (file_path, offset, length)!')
                else:
                    print(e)
                continue
        elif commands[0] == 'rm':
            try:
                demo.rm(commands[1])
                continue
            except Exception as e:
                if type(e) == IndexError:
                    print('Please give parameter (file_path)!')
                else:
                    print(e)
                continue
        elif commands[0] == 'rmdir':
            try:
                demo.rmdir(commands[1])
                continue
            except Exception as e:
                if type(e) == IndexError:
                    print('Please give parameter (dir_path)!')
                else:
                    print(e)
                continue
        elif commands[0] == 'getPartitionLocations':
            try:
                demo.getPartitionLocations(commands[1])
                continue
            except Exception as e:
                if type(e) == IndexError:
                    print('Please give parameter (file_path)!')
                else:
                    print(e)
                continue
        
        elif commands[0] == 'quit' or commands[0] == 'break' or commands[0] == 'logout':
            print('Bye!')
            break
        else:
            print("Please input the correct commands")
            continue
    demo.endconn()
# elif commands[0] == 'readPartition':
#             try:
#                 demo.getPartitionLocations(commands[1], command[2])
#                 continue
#             except Exception as e:
#                 if type(e) == IndexError:
#                     print('Please give parameter (file_path, partition_num)!')
#                 else:
#                     print(e.message)
#                 continue

In [4]:
YELP_ANALYSIS_APP()

Welcome to the YELP ANALYSIS system!
Now you are in the EDFS. Please write your commands below!
Initialization success!!!
>>> ls /user/johns/a

>>> ls /user/johns/b
business_2022_10_30.json	business_2022_10_30_1.json
>>> mkdir /users/js/a
The directory /users/js/a  is successfully made!
>>> ls /
user	users
>>> cat /user/johns/b/business_2022_10_30.json
Please give parameters (file_path, offset, length)!
>>> cat /user/johns/b/business_2022_10_30.json 37710 20
               business_id                                         name  \
0   BqGM1NCltdaweP_WB1Klhg                                     Bisteces   
1   BqGWgwUAUi9rtIh_bKNMNg                  Saguaro Landscaping & Pools   
2   BqHX49ixrSuPoMaZ9Lzz1g                                  The Nailery   
3   BqIbxfNBEHbjHuYb2Y2ZvQ                                 AA Auto Care   
4   BqKFAnGL8MLl7NtYNUhSYg                             Boîte à Fromages   
5   BqKq0tKOAEvyHdjGtWcEHQ                              The Gaming Goat   
6   BqPDyBgR

In [16]:
'''
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
x = np.array([1,2,3,4,5,6,7,8])
def myfunc(a,b,c):
    y = (x-1)**2+2
    plt.scatter(x,y,c='k',lw=3,s=60)
    r = (x-a)**b+c
    cost = np.sum(r-y)
    plt.plot(x,r,c='yellow')
    plt.scatter(0,cost,c='blue')
    plt.show()
    return r

myy = interactive(myfunc,a=(-5,5,0.5),b=[1,2,3,4,5],c=(-4,4,0.5))
display(myy)
'''

interactive(children=(FloatSlider(value=0.0, description='a', max=5.0, min=-5.0, step=0.5), Dropdown(descripti…